In [ ]:
# pip install sentence-transformers
# pip show chromadb

In [2]:
import chromadb
from chromadb import PersistentClient # DB 접속자 역할
print(chromadb.__file__) # 설치된 chromadb 패키지의 경로 출력 (디버깅/확인용)

# chroma는 DuckDB 기반의 벡터 저장소로 작동
# PersistentClient: 데이터가 디스크에 저장되어 세션이 종료되어도 유지되도록 설정
client = PersistentClient(path=".chroma") # ".chroma" 폴더에 DB 파일 저장
!pwd # 현재 작업 디렉토리 출력
!ls -a # 현재 디렉토리의 파일 목록 출력 (.chroma 폴더가 생성되었는지 확인 가능)

c:\Users\acorn\anaconda3\Lib\site-packages\chromadb\__init__.py


'pwd'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [ ]:
# ChromaDB 컬렉션에 문서를 추가하기 전에,*문자열 데이터를 벡터로 변환하는 과정과 벡터 간의 유사성을 확인하는 단계. 
# 이는 RAG(검색 증강 생성) 파이프라인의 임베딩 및 유사도 측정.


# collection: RDBMS Table과 유사
# get_or_create_collection: 이미 존재하면 가져오고, 없으면 새로 생성
# 기본 내장 모델: all-MiniLM-L6-v2 (텍스트를 벡터로 변환하는 기본 모델)
collection = client.get_or_create_collection("test")
print(collection, ' ', collection.id)

# 문서를 벡터화해서 DB에 저장
texts = ['Hello world', 'Chroma is cool'] # 벡터화할 원본 텍스트 데이터
ids = ['doc1', 'doc2'] # 각 문서의 고유 식별자 (DB의 기본 키 역할)
metas = [{'source':'greeting'}, {'source':'statement'}] # 텍스트에 대한 부가 정보 (메타데이터)

embedding_fn = collection._embedding_function # 컬렉션에 설정된 임베딩 모델(여기서는 all-MiniLM-L6-v2)을 가져옴. 임베딩 함수 반환 (텍스트 -> 벡터화 수행)
embeddings = embedding_fn(texts) # texts 리스트를 입력으로 받아 벡터 리스트를 반환

# collection: RDB의 테이블(Table)과 같은 개념으로, 벡터와 텍스트, 메타데이터가 저장되는 공간.
# embedding_fn의 역할: 트랜스포머 기반 임베딩 모델을 사용하여 텍스트를 컴퓨터가 이해할 수 있는 수치형 언어 (벡터)로 변환. all-MiniLM-L6-v2 모델을 사용하면 벡터의 차원은 384가 됨.


# 임베딩 벡터 속성 확인
print(type(embeddings), len(embeddings), len(embeddings[0]))    # 'list' 타입, 2개의 문서에 대한 벡터, 각 벡터의 차원 수는 384
print(embeddings[0][:5]) # [-0.03447731  0.03102319  0.00673492  0.02610894 -0.03936205]

for i, vector in enumerate(embeddings):
    print('-' * 50)
    print(f'문서:{texts[i]}')
    print(f'임베딩 벡터 앞 5개: {vector[:5]}')
    print(f'차원 수: {len(vector)}')     # 모든 벡터는 동일한 차원(384)을 가짐
print('~' * 50)

# 차원 수 (Dimension): 출력된 `384`는 벡터 공간의 크기.이 차원 수가 클수록 텍스트의 의미적 뉘앙스를 더 잘 담아낼 수 있음.
# 임베딩 벡터: 각 텍스트가 의미적 특징을 담고 있는 384차원 실수 배열(벡터)로 성공적으로 변환되었음을 확인.


# 코사인 유사도 측정 (검색의 핵심 원리)_두 문장 
from sklearn.metrics.pairwise import cosine_similarity # cosine_similarity: 두 벡터 사이의 각도(방향) 유사도를 측정
sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
print(f'코사인 유사도: {sim:.5f}')


# 코사인 유사도 (Cosine Similarity): RAG 시스템의 검색(Retrieval)단계에서 가장 중요한 개념.

# [Image of Cosine Similarity between two vectors]
# 벡터 공간에서 두 벡터가 얼마나 같은 방향을 가리키는지를 나타내며, 1에 가까울수록 의미가 매우 유사함을 뜻.
# 작동 원리: `Hello world`와 `Chroma is cool`은 의미적으로 관련성이 낮으므로, 이 두 벡터 사이의 유사도 값은 0에 가까운 작은 양수가 나올 것. RAG는 사용자의 질문 벡터와 이 유사도를 비교하여 가장 높은 유사도를 가진 문서를 검색.
# 코드는 텍스트 데이터를 수치화하고, 벡터 공간에서 의미를 비교하는 벡터 DB의 핵심 기능을 보임.

Collection(name=test)   d28be5de-a002-4cad-8724-2ce4eb64a506


C:\Users\acorn\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:15<00:00, 5.21MiB/s]


<class 'list'> 2 384
[-0.03447731  0.03102319  0.00673492  0.02610894 -0.03936205]
--------------------------------------------------
문서:Hello world
임베딩 벡터 앞 5개: [-0.03447731  0.03102319  0.00673492  0.02610894 -0.03936205]
차원 수: 384
--------------------------------------------------
문서:Chroma is cool
임베딩 벡터 앞 5개: [-0.11315749 -0.00511693 -0.01389765 -0.02509307 -0.04774464]
차원 수: 384
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
코사인 유사도: 0.05800


In [ ]:
# 벡터 데이터베이스인 ChromaDB에 문서를 저장(Indexing)하고, 저장된 데이터를 조회(Get)하는 과정. 이는 RAG 시스템 구축의 가장 기본이 되는 단계.
# RDBMS의 `INSERT INTO`와 `SELECT` 명령어의 벡터 DB 버전으로 이해하시면 매우 직관적.


# collection에 문서 + 벡터 + 메타데이터 저장
collection.add(
    documents=texts,   # 원본 텍스트 데이터 (문서 내용)
    embeddings=embeddings, # 텍스트를 변환한 384차원 벡터
    metadatas=metas,   # 텍스트에 대한 부가 정보 (소스, 날짜 등)
    ids=ids,           # 고유 식별자 ('doc1', 'doc2')
    # uris=['https://example.com/test1'] # 선택적으로 문서의 출처 URI를 저장할 수도 있음
)
print(collection)
# * RDB 비교 (INSERT): `collection.add()` 함수는 RDBMS에서 `INSERT INTO table_name (id, document, embedding, metadata) VALUES (...)` 와 동일한 역할.
# * 핵심 기능: `add()`를 통해 텍스트(사람이 읽는 정보)와 벡터(기계가 계산하는 정보)가 영구적으로 연결되어 DB에 저장.
# collection에 저장된 자료 조회
# `collection.get()`: RDB의 `SELECT * FROM table_name`과 유사. 컬렉션에 저장된 모든 데이터를 조회.
# `include` 파라미터의 중요성:
#       1. `get()` 함수는 기본적으로 대용량인 `embeddings`는 반환하지 않습니다.
#       2. 만약 벡터를 포함시키지 않으면 네트워크 대역폭과 메모리를 절약할 수 있습니다.
#       3. 여기서는 원본 텍스트와 메타데이터만 잘 저장되었는지 확인하기 위해 명시적으로 요청했습니다.
results = collection.get(include=['documents', 'metadatas'])    # include=['documents', 'metadatas']: 벡터(embeddings)는 제외하고 텍스트와 메타데이터만 조회
print(results)

# 조회된 결과를 zip 함수로 묶어 문서, 메타데이터, ID 순서로 출력
for doc, meta, id in zip(results['documents'], results['metadatas'], results['ids']):
    print('-' * 50)
    print(f'ids: {id}')
    print(f'document: {doc}')
    print(f'metadata: {meta}')
print('~' * 50)

print('저장된 문서 id 목록: ', collection.get()['ids'])
# 확인 목적: `add()` 명령이 성공적으로 실행되어, 이전에 정의했던 고유 ID와 원본 텍스트, 그리고 메타데이터가 정확히 매핑되어 저장되었음을 눈으로 확인하는 과정.
result_vec = collection.get(include=['embeddings'])

# 첫번째 문서의 임베딩 벡터 자료 출력
first_embedding = result_vec['embeddings'][0]
# 벡터의 실제 값과 차원 수를 다시 확인
print('임베딩 벡터 차원 수: ', first_embedding[:2], ' ', len(first_embedding))
print()

for id_, embed in zip(result_vec['ids'], result_vec['embeddings']):
    print(f'id: {id_}')
    print(f'임베딩 앞 5개: {embed[:5]}')

# `collection.get(include=['embeddings'])`: 벡터 DB의 핵심 저장소인 벡터 데이터만 가져와서 확인.
# 차원 수 재확인: `len(first_embedding)`을 통해 이전에 확인했던 384 차원이 DB에 정확하게 저장되었음을 다시 한번 입증
# RAG 시스템에서 "내가 검색할 자료를 성공적으로 벡터 DB에 넣어놨는가?"를 확인하는 필수적인 검증 단계

Collection(name=test)
{'ids': ['doc1', 'doc2'], 'embeddings': None, 'documents': ['Hello world', 'Chroma is cool'], 'uris': None, 'included': ['documents', 'metadatas'], 'data': None, 'metadatas': [{'source': 'greeting'}, {'source': 'statement'}]}
--------------------------------------------------
ids: doc1
document: Hello world
metadata: {'source': 'greeting'}
--------------------------------------------------
ids: doc2
document: Chroma is cool
metadata: {'source': 'statement'}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
저장된 문서 id 목록:  ['doc1', 'doc2']
임베딩 벡터 차원 수:  [-0.03447731  0.03102319]   384

id: doc1
임베딩 앞 5개: [-0.03447731  0.03102319  0.00673492  0.02610894 -0.03936205]
id: doc2
임베딩 앞 5개: [-0.11315749 -0.00511693 -0.01389765 -0.02509307 -0.04774464]


RDBMS는 관계형 데이터베이스 관리 시스템(Relational Database Management System)

1. RDBMS란?

RDBMS는 데이터를 테이블(Table)형태로 구성하며, 이 테이블들은 사전 정의된 관계(Relationship)에 따라 논리적으로 연결.

* 관계형 (Relational): 데이터를 테이블 형태로 구성하고, 각 테이블의 열(Column)과 행(Row)을 이용하여 데이터 간의 관계를 명확하게 정의.
* 데이터베이스 (Database): 데이터를 저장하고 유지하는 시스템.
* 관리 시스템 (Management System): 데이터의 저장, 검색, 수정, 삭제 등의 작업을 효율적으로 수행할 수 있도록 관리해주는 소프트웨어.

2. RDBMS의 주요 특징과 명령어

RDBMS는 데이터를 정형화된 형태로 관리하며, 주로 SQL (Structured Query Language) 언어를 사용하여 데이터를 조작.

| 구분 | RDBMS의 개념 | 벡터 DB의 유사 개념 (ChromaDB) |

| 데이터 저장 단위       | 테이블 (Table)        | 컬렉션 (Collection) |
| 데이터 구조           | 행(Row), 열(Column)   | 벡터, 문서(텍스트), 메타데이터 |
| 삽입/저장 명령어       | `INSERT INTO`        | `collection.add()` (데이터 인덱싱) |
| 조회 명령어           | `SELECT`             | `collection.get()` (단순 조회) |
| 검색 명령어           | `SELECT` + `WHERE` 조건 | `collection.query()` (유사도 검색) |

3. 예시

* `INSERT INTO`의 역할: 새로운 행(Row)을 테이블에 추가하여 데이터를 저장. (예: 새로운 사용자 정보 입력)
* `SELECT`의 역할: 테이블에서 원하는 조건에 맞는 데이터를 검색하고 추출. (예: 특정 이름의 사용자 정보 조회)

정형 데이터 관리 시스템의 개념 -  ChromaDB의 `collection.add()`가 벡터와 메타데이터를 저장하는 `INSERT INTO`와 같고, `collection.query()`가 유사도 기반의 `SELECT`와 같다고 이해하면 RAG 시스템의 데이터 흐름을 쉽게 파악 O.

In [ ]:
# RAG(검색 증강 생성) 시스템의 핵심인 검색(Retrieval) 단계를 ChromaDB를 이용해 구현한 것. 저장된 문서 중에서 사용자의 질문과 의미적으로 가장 유사한 문서를 찾아내는 과정.
# 벡터 기반 유사도 검색 (쿼리 벡터화)
query_text = "Chroma에 대해 설명해줘" # 검색용 질문(사람의 언어)
# 쿼리도 반드시 벡터로 변환해야 함.
# embedding_fn: all-MiniLM-L6-v2 모델을 사용하여 텍스트를 384차원 벡터로 변환
query_embedding = embedding_fn([query_text])[0] # 문장 -> 벡터화 (기계의 언어)

# 필수 단계: 벡터 데이터베이스는 텍스트를 직접 이해하지 못하고 벡터만 비교할 수 있음. 따라서 검색을 수행하려면 사용자의 질문(`query_text`)도 이전에 문서 저장 시 사용했던 것과 동일한 임베딩 모델(`embedding_fn`)을 사용하여 벡터(`query_embedding`)로 변환해야 함.

# Chroma에 저장된 자료 중에서 유사 자료 검색(`collection.query()`)
search_result = collection.query(
    query_embeddings=[query_embedding], # 질문을 벡터로 바꿈 결과로 대입 ( # 변환된 질문 벡터를 입력 )
    n_results=2, # 유사도가 높은 자료 2개 반환
    include=['documents', 'metadatas', 'distances']# 검색 결과에 포함시킬 항목 지정
)
print(search_result)

# `collection.query()`: 벡터 DB에게 "이 질문 벡터와 가장 가까운 문서를 찾아줘"라고 명령하는 함수.
# * 작동 원리: ChromaDB는 질문 벡터와 저장된 모든 문서 벡터 사이의 유사도 거리(Distance)를 계산.
# * 내부적으로는 HNSW(Hierarchical Navigable Small World)와 같은 근접 이웃 탐색 알고리즘을 사용하여 수많은 벡터를 효율적으로 비교.
# * `n_results=2`: 저장된 문서(`Hello world`, `Chroma is cool`) 중에서 가장 유사한 2개의 문서를 순서대로 반환하도록 설정.
# * `distances`: 질문 벡터와 문서 벡터 사이의 거리 값. 값이 작을수록 (0에 가까울수록) 유사도가 높음을 의미.


# 결과 출력
for i, (doc, meta, dist) in enumerate(zip(
    search_result['documents'][0],
    search_result['metadatas'][0],
    search_result['distances'][0],)):
    print(f'\n결과 {i + 1}')
    print(f'document: {doc}')
    print(f'metadata: {meta}')
    print(f'distance(유사도 거리): {dist:.4f}')

# 결과 분석:
#       * `query_text`는 "Chroma에 대해 설명해줘"입니다.
#       * 저장된 문서에는 "Hello world"와 "Chroma is cool"이 있습니다.
#       * 'Chroma'라는 키워드를 포함하고 있는 "Chroma is cool" 문서가 질문과 의미적으로 훨씬 가깝기 때문에, 이 문서가 가장 짧은 거리(`dist`가 가장 작은 값)로 첫 번째 결과(결과 1)로 반환될 것.
# 이 단계가 성공적으로 끝나면, RAG 시스템은 검색된 문서 내용(`document: Chroma is cool`)을 LLM(예: Gemini-2.5-flash)에게 전달하여 최종 답변을 생성.

{'ids': [['doc2', 'doc1']], 'embeddings': None, 'documents': [['Chroma is cool', 'Hello world']], 'uris': None, 'included': ['documents', 'metadatas', 'distances'], 'data': None, 'metadatas': [[{'source': 'statement'}, {'source': 'greeting'}]], 'distances': [[0.724317729473114, 2.1710548400878906]]}

결과 1
document: Chroma is cool
metadata: {'source': 'statement'}
distance(유사도 거리): 0.7243

결과 2
document: Hello world
metadata: {'source': 'greeting'}
distance(유사도 거리): 2.1711
